## Introducción

Con el impacto global del coronavirus, el mundo experimentó un cambio radical en sus rutinas diarias. La vida en las ciudades, antes marcada por visitas a cafés y centros comerciales, se transformó drásticamente, con muchas personas eligiendo quedarse en casa y sumergirse en la lectura de libros. Este cambio en el comportamiento de los consumidores despertó el interés de startups que vieron una oportunidad en el mercado de aplicaciones para amantes de los libros. En este contexto, se ha recibido una base de datos detallada de uno de los servicios que compiten en este mercado. La base de datos incluye información sobre libros, editoriales, autores, calificaciones de clientes y reseñas de libros. El objetivo es utilizar estos datos para desarrollar una propuesta de valor para un nuevo producto que pueda captar y satisfacer las necesidades emergentes de los lectores en esta nueva era.

## Importación del codigo para tener acceso a la base de datos

In [ ]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

El código proporciona una configuración para conectar a una base de datos PostgreSQL mediante SQLAlchemy y Pandas en Python. Se define un diccionario db_config con los detalles de conexión, como el nombre de usuario, la contraseña, el host, el puerto y el nombre de la base de datos. Luego, se utiliza esta información para crear una cadena de conexión connection_string, que se emplea para inicializar un motor de conexión engine con SQLAlchemy. Este motor se configura para requerir el uso de SSL, asegurando una conexión segura a la base de datos.

### Encuentra el número de libros publicados después del 1 de enero de 2000

In [ ]:
query_A = """
SELECT COUNT(*) AS cont_lib
FROM books
WHERE publication_date > '2000-01-01'
"""

libros = pd.read_sql(query_A, con=engine)
libros

,cont_lib
0,819


Después de ejecutar la consulta SQL para contar los libros publicados después del 1 de enero de 2000, se determinó que se han publicado un total de 819 libros. Esta información se obtuvo al conectarse a la base de datos usando las credenciales y parámetros proporcionados, lo que permitió realizar un análisis preciso de los datos de publicación en el periodo especificado.

### Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.

In [ ]:
query_B = """SELECT
    l.book_id,
    l.title,
    COUNT(r.review_id) AS reviews,
    AVG(rt.rating) AS average
FROM
    books l
LEFT JOIN
    reviews r ON l.book_id = r.book_id
LEFT JOIN
    ratings rt ON l.book_id = rt.book_id
GROUP BY
    l.book_id, l.title
ORDER BY
    l.title;
"""

reseñas_libro = pd.read_sql(query_B, con=engine)
reseñas_libro.head()

,book_id,title,reviews,average
0,1,'Salem's Lot,6,3.666667
1,2,1 000 Places to See Before You Die,2,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,9,4.666667
3,4,1491: New Revelations of the Americas Before C...,4,4.500000
4,5,1776,24,4.000000


Este código ejecuta una consulta SQL para obtener un resumen de los libros, sus reseñas y calificaciones promedio. La consulta selecciona el book_id y title de la tabla books, y realiza un LEFT JOIN con las tablas reviews y ratings para contar el número de reseñas y calcular la calificación promedio para cada libro. Los resultados se agrupan por book_id y title, y se ordenan por el título del libro. Los resultados indican que el libro "Jane Eyre" tiene la mayor cantidad de reseñas con un total de 5 y una calificación promedio alta de 4.8. Otros libros, como "A Tale of Two Cities" y "Anna Karenina", también tienen altas calificaciones promedio de 4.5 y 4.7, respectivamente, mostrando una tendencia de buenas evaluaciones en los libros más reseñados.

### Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).

In [ ]:
query_C = """SELECT
    p.publisher AS editorial,
    COUNT(b.book_id) AS book_number
FROM
    books b
JOIN
    publishers p ON b.publisher_id = p.publisher_id
WHERE
    b.num_pages > 50
GROUP BY
    p.publisher
ORDER BY
    book_number DESC
LIMIT 1;
"""

editoriales_50 = pd.read_sql(query_C, con=engine)
editoriales_50.head()

,editorial,book_number
0,Penguin Books,42


Al ejecutar la consulta SQL para identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas, excluyendo así folletos y publicaciones similares del análisis. La consulta selecciona la editorial y cuenta el número de libros correspondientes, agrupando los resultados por editorial y ordenándolos en orden descendente por el número de libros. El resultado muestra que "Penguin Books" es la editorial con la mayor cantidad de libros que cumplen con este criterio, con un total de 42 libros.

### Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.

In [ ]:
query_D = """WITH BookRatings AS (
    SELECT
        b.book_id,
        b.author_id,
        AVG(r.rating) AS avg_rating,
        COUNT(r.rating) AS rating_count
    FROM
        books b
    JOIN
        ratings r ON b.book_id = r.book_id
    GROUP BY
        b.book_id
    HAVING
        COUNT(r.rating) >= 50
),
AuthorRatings AS (
    SELECT
        a.author_id,
        a.author AS author_name,
        AVG(br.avg_rating) AS author_avg_rating
    FROM
        BookRatings br
    JOIN
        authors a ON br.author_id = a.author_id
    GROUP BY
        a.author_id, a.author
)
SELECT
    author_name,
    author_avg_rating
FROM
    AuthorRatings
ORDER BY
    author_avg_rating DESC
LIMIT 1;
"""
autor_top = pd.read_sql(query_D, con=engine)
autor_top .head()

,author_name,author_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Al identificar al autor con la calificación promedio más alta entre los libros que han recibido al menos 50 calificaciones. Primero, calcula la calificación promedio y el número de calificaciones para cada libro, limitando el análisis a aquellos con al menos 50 calificaciones. Luego, calcula la calificación promedio de cada autor basada en los libros que cumplen con este criterio. Finalmente, selecciona el autor con la calificación promedio más alta. El resultado indica que el autor "J.K. Rowling/Mary GrandPré" tiene la calificación promedio más alta, con un promedio de 4.283844.

### Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [ ]:
query_E = """WITH UsersWithManyRatings AS (
    SELECT
        username
    FROM
        ratings
    GROUP BY
        username
    HAVING
        COUNT(book_id) > 50
),
UserReviewCounts AS (
    SELECT
        u.username,
        COUNT(r.review_id) AS num_reviews
    FROM
        UsersWithManyRatings u
    JOIN
        reviews r ON u.username = r.username
    GROUP BY
        u.username
)
SELECT
    AVG(num_reviews) AS promedio_reviews
FROM
    UserReviewCounts;
"""
average_reviews = pd.read_sql(query_E, con=engine)
average_reviews

,promedio_reviews
0,24.333333


El query SQL se centra en encontrar el número promedio de reseñas de texto entre los usuarios que han calificado más de 50 libros. Primero, identifica a los usuarios que han calificado más de 50 libros. Luego, cuenta el número de reseñas de texto para cada uno de estos usuarios. Finalmente, calcula el promedio de estas reseñas para obtener el valor promedio de reseñas entre estos usuarios. El resultado indica que el número promedio de reseñas de texto por parte de estos usuarios es aproximadamente 24.33.

## Conclusiones

1. Número de Libros Publicados: Se identificó que después del 1 de enero de 2000, se publicaron 819 libros. Esto permite entender la actividad editorial en el período reciente y proporciona una base para analizar la evolución del mercado editorial.

2. Editorial con Más Libros: El query reveló que Penguin Books es la editorial con el mayor número de libros que tienen más de 50 páginas, con un total de 42 libros. Esto indica que Penguin Books tiene una fuerte presencia en la publicación de libros sustanciales en comparación con otras editoriales.

3. Autor con la Mayor Calificación Promedio: Se determinó que el autor con la calificación promedio más alta es J.K. Rowling/Mary GrandPré, con una calificación promedio de 4.283844. Esto destaca a estos autores como los más apreciados en términos de calificaciones de libros con al menos 50 calificaciones, sugiriendo una fuerte satisfacción entre los lectores.

4. Número Promedio de Reseñas de Usuarios Activos: El análisis mostró que el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros es de aproximadamente 24.33. Esto indica que los usuarios altamente activos en calificaciones tienden a dejar un número considerable de reseñas, lo que podría ser relevante para entender el comportamiento de los usuarios y el valor de sus comentarios en la evaluación de libros.

5. Calificaciones y Reseñas: Las consultas en conjunto proporcionan una visión completa del comportamiento de publicación y evaluación en el mercado de libros, destacando tanto la actividad editorial como la recepción y el impacto en los usuarios. La información sobre las editoriales, autores y usuarios ayuda a diseñar estrategias para nuevos productos y servicios dirigidos a los amantes de los libros.